In [0]:
bronze_df = spark.read.table("workspace.bronze.sales_orders_raw")

bronze_df.printSchema()


In [0]:
bronze_df.select("value").limit(1).display()


In [0]:
from pyspark.sql.types import (
    StructType, StructField,
    StringType, DoubleType, TimestampType
)


In [0]:
from pyspark.sql.functions import col, get_json_object

bronze_df.select(
    get_json_object(col("value"), "$").alias("raw_json")
).limit(3).display()


In [0]:
%sql
CREATE TABLE IF NOT EXISTS workspace.silver.sales_orders_clean (
    order_id BIGINT,
    customer_id STRING,
    number_of_line_items STRING,
    ordered_products ARRAY<STRUCT<
        curr: STRING,
        id: STRING,
        name: STRING,
        price: BIGINT,
        qty: BIGINT,
        unit: STRING
    >>,
    order_timestamp TIMESTAMP,
    file_path STRING,
    ingestion_timestamp TIMESTAMP
)
USING DELTA;


In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, expr, from_unixtime

# 1. Type casting (defensive)
silver_cast_df = (
    parsed_df
    .withColumn("order_id", expr("try_cast(order_id as bigint)"))
    .withColumn(
        "order_timestamp",
        from_unixtime(expr("try_cast(order_datetime as bigint)"))
    )
    .drop("order_datetime")
)

# 2. Filter invalid business keys
silver_valid_df = silver_cast_df.filter(col("order_id").isNotNull())

# 3. Deduplicate (THIS IS CRITICAL)
window_spec = Window.partitionBy("order_id").orderBy(
    col("ingestion_timestamp").desc()
)

silver_dedup_df = (
    silver_valid_df
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn")
)

# 4. MERGE MUST USE silver_dedup_df
silver_dedup_df.createOrReplaceTempView("sales_orders_updates")

spark.sql("""
MERGE INTO workspace.silver.sales_orders_clean tgt
USING sales_orders_updates src
ON tgt.order_id = src.order_id

WHEN MATCHED THEN
  UPDATE SET *

WHEN NOT MATCHED THEN
  INSERT *
""")


In [0]:
%sql
select* from workspace.silver.sales_orders_clean tgt

In [0]:
%sql
-- 1. Row count
SELECT COUNT(*) 
FROM workspace.silver.sales_orders_clean;


In [0]:
%sql
-- 2. Business key uniqueness
SELECT COUNT(*) AS total_rows,
       COUNT(DISTINCT order_id) AS distinct_orders
FROM workspace.silver.sales_orders_clean;
